In [22]:
import numpy as np 
import pandas as pd

import sklearn.metrics as metrics

In [23]:
data_file = f"data/5_mr_50_cond/simulated_noNoise.txt"
grn_file = f"data/5_mr_50_cond/bipartite_GRN.csv"

data = pd.read_csv(data_file, sep = "\t", header = 0)
grn_df = pd.read_csv(grn_file, sep = ",", header = None)
grn_true = pd.Series(zip(*map(grn_df.get, grn_df)), name='edges').to_frame()
grn_true['class'] = 1

# compute pairwise person correlations between genes and TFs
corr_mat = data.corr(method="pearson").abs().iloc[100:,:100]
corr_dict = corr_mat.unstack().to_dict()

grn_pred = pd.concat([pd.Series(corr_dict.keys(), name='edges'),  pd.Series(corr_dict.values(), name='corr')], axis=1)

grn_eval = pd.merge(grn_pred, grn_true, on='edges', how="left")
grn_eval['class'] = grn_eval['class'].fillna(int(0))

grn_eval.to_csv(r"results/5_mr_50_cond/top_corr.csv")


In [24]:
# Antonio: splitting the cells because I am getting an error in this cell
print(metrics.roc_auc_score(grn_eval['class'], grn_eval['corr']))

for i in range(100):
    grn_eval_gene = grn_eval.iloc[i::100,:]
    print(metrics.roc_auc_score(grn_eval_gene['class'], grn_eval_gene['corr']))


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
# data_file = f"data/40_mr_50_cond/simulated_noNoise.txt"
# grn_file = f"data/40_mr_50_cond/bipartite_GRN.csv"
#
# data = pd.read_csv(data_file, sep = "\t", header = None)
# grn_df = pd.read_csv(grn_file, sep = ",", header = None)
# grn_true = pd.Series(zip(*map(grn_df.get, grn_df)), name='edges').to_frame()
# grn_true['class'] = 1
#
# corr_mat = data.corr(method="pearson").abs().iloc[100:,:100]
# corr_dict = corr_mat.unstack().to_dict()
#
# grn_pred = pd.concat([pd.Series(corr_dict.keys(), name='edges'),  pd.Series(corr_dict.values(), name='corr')], axis=1)
#
# grn_eval = pd.merge(grn_pred, grn_true, on='edges', how="left")
# grn_eval['class'] = grn_eval['class'].fillna(int(0))
#
# grn_eval.to_csv(r"results/40_mr_50_cond/top_corr.csv")
#
# print(metrics.roc_auc_score(grn_eval['class'], grn_eval['corr']))
#
# for i in range(100):
#     grn_eval_gene = grn_eval.iloc[i::100,:]
#     print(metrics.roc_auc_score(grn_eval_gene['class'], grn_eval_gene['corr']))
#

In [ ]:
# data_file = f"data/100_mr_50_cond/simulated_noNoise.txt"
# grn_file = f"data/100_mr_50_cond/bipartite_GRN.csv"
#
# data = pd.read_csv(data_file, sep = "\t", header = None)
# grn_df = pd.read_csv(grn_file, sep = ",", header = None)
# grn_true = pd.Series(zip(*map(grn_df.get, grn_df)), name='edges').to_frame()
# grn_true['class'] = 1
#
# corr_mat = data.corr(method="pearson").abs().iloc[100:,:100]
# corr_dict = corr_mat.unstack().to_dict()
#
# grn_pred = pd.concat([pd.Series(corr_dict.keys(), name='edges'),  pd.Series(corr_dict.values(), name='corr')], axis=1)
#
# grn_eval = pd.merge(grn_pred, grn_true, on='edges', how="left")
# grn_eval['class'] = grn_eval['class'].fillna(int(0))
#
# grn_eval.to_csv(r"results/100_mr_50_cond/top_corr.csv")
#
# print(metrics.roc_auc_score(grn_eval['class'], grn_eval['corr']))
#
# for i in range(100):
#     grn_eval_gene = grn_eval.iloc[i::100,:]
#     print(metrics.roc_auc_score(grn_eval_gene['class'], grn_eval_gene['corr']))